# Table of Contents

1. [Building Causal Models](#Building-Causal-Models)
2. [Model Analysis](#Model-Analysis)
    1. [Causal Effect of Changing Neighborhoods](#Causal-Effect-on-ROI-of-Changing-Neighborhoods---Should-we-choose-North-or-South?)
    2. [Causal Effect of Different BPBs](#Causal-Effect-on-ROI-of-Different-BPBs)
    3. [Given we want a certain Bed and Baths, where should we buy?](#Given-we-want-a-certain-Bed-and-Baths,-where-should-we-buy?)
    4. [Given we want to buy in a certain area, what beds and baths should we buy?](#Given-we-want-to-buy-in-a-certain-area,-what-beds-and-baths-should-we-buy?)
3. [Property Examples](#Property-Examples)
4. [Next Steps](#Next-Steps)

In [1]:
import pyro
import pyro.distributions as dist
from pyro.infer import Importance, EmpiricalMarginal
from statistics import mean
import torch
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
pyro.set_rng_seed(101)

In [2]:
df = pd.read_csv("data/cleansed_data.csv")

# Building Causal Models

### Fitting Factor Effects Models for Rent and Zestimate
- In order to get the parameters for the Rent and Zestimate calculations we use factor effects models(aka ANOVA models)

Taken from [this](https://pythonfordatascience.org/anova-2-way-n-way/) tutorial

In [3]:
rent_model = ols('Rent ~ C(BPB)', df).fit()

# Seeing if the overall model is significant
print(f"Overall model F({rent_model.df_model: .0f},{rent_model.df_resid: .0f}) = {rent_model.fvalue: .3f}, p = {rent_model.f_pvalue: .4f}")

Overall model F( 4, 200) =  13.359, p =  0.0000


Model is signficant

In [4]:
res = sm.stats.anova_lm(rent_model, typ=2)
rent_model_se = (res.sum_sq.Residual / res.df.Residual)**0.5
res

,sum_sq,df,F,PR(>F)
C(BPB),242221.823818,4.0,13.359283,1.148343e-09
Residual,906567.424962,200.0,NaN,NaN


- Do OLS model for Zestimates
- Get Neighborhood Marginal
- Get BPB marginal
- Get ROI function in the code

In [5]:
zest_model = ols('Zest ~ C(BPB) + C(Neigh)', df).fit()

# Seeing if the overall model is significant
print(f"Overall model F({zest_model.df_model: .0f},{zest_model.df_resid: .0f}) = {zest_model.fvalue: .3f}, p = {zest_model.f_pvalue: .4f}")

Overall model F( 5, 199) =  8.702, p =  0.0000


In [6]:
res = sm.stats.anova_lm(zest_model, typ=2)
zest_model_se = (res.sum_sq.Residual / res.df.Residual)**0.5
res

,sum_sq,df,F,PR(>F)
C(BPB),9.832538e+11,4.0,0.417948,7.955891e-01
C(Neigh),2.399361e+13,1.0,40.795532,1.170317e-09
Residual,1.170405e+14,199.0,NaN,NaN


### Functions for Calculating ROI
- below are the functions and hyperparameters for calculating ROI

In [7]:
# ROI Consts
INFLATION_RATE = 0.028
MORTGAGE_RATE = 0.036
NUM_YEARS = 15
DOWN_PAYMENT = 0.20


def calculate_monthly(P, mortgage_rate, num_years):
    n = num_years * 12
    monthly_i = mortgage_rate / 12
    numerator = monthly_i * (1 + monthly_i) ** n
    denominator = ((1 + monthly_i) ** n) - 1
    return P * numerator / denominator


def airbnb_income(price, inflation_rate, num_years):
    total = 0
    for year_number in range(num_years):
        curr_inflation = (1 + inflation_rate) ** year_number
        total += (price * curr_inflation) * 12
    return total


def roi(zestimate, inflation_rate, mortgage_rate, num_years, rental_price, down_payment_percent):
    down_payment = zestimate * down_payment_percent
    P = zestimate * (1 - down_payment_percent)

    incurred_cost = calculate_monthly(
        P, mortgage_rate, num_years) * 12 * num_years + down_payment

    income = airbnb_income(price=rental_price,
                           inflation_rate=inflation_rate,
                           num_years=num_years)

    return (income - incurred_cost) / incurred_cost

### Model 1

- Simpler causal model without exogenous variables

In [8]:
Neigh_alias = ['Neigh_North', 'Neigh_South']
BPB_alias = [2, 3, 4, 5, 6]

north_prob = float(len(df[df['Neigh'] == 'Neigh_North']))/len(df)
bpb_prob = [(df.BPB == a).count()/len(df) for a in BPB_alias]


Neigh_prob = torch.tensor([north_prob, 1 - north_prob])
BPB_prob = torch.tensor(bpb_prob)


def model():
    Neigh = pyro.sample("Neigh", dist.Categorical(probs=Neigh_prob))
    BPB = pyro.sample("BPB", dist.Categorical(probs=BPB_prob))

    bpb_converted = BPB_alias[BPB]
    neigh_converted = Neigh_alias[Neigh]
    rent_pred = rent_model.predict(pd.DataFrame({'BPB': [bpb_converted]}))[0]
    zest_pred = zest_model.predict(pd.DataFrame(
        {'BPB': [bpb_converted], 'Neigh': [neigh_converted]}))[0]

    Rent = pyro.sample("Rent", dist.Delta(torch.tensor(rent_pred)))
    Zest = pyro.sample("Zest", dist.Delta(torch.tensor(zest_pred)))

    ROI = pyro.sample("ROI", dist.Delta(roi(zestimate=Zest,
                                            rental_price=Rent,
                                            inflation_rate=INFLATION_RATE,
                                            mortgage_rate=MORTGAGE_RATE,
                                            num_years=NUM_YEARS,
                                            down_payment_percent=DOWN_PAYMENT)))

    return {'Neigh': Neigh, 'BPB': BPB, 'Rent': Rent, 'Zest': Zest, 'ROI': ROI}

### Model 2 - SCM

\begin{align}
Nb &= p(BPB=2)=p_2, p(BPB=3)=p_3, p(BPB=4)=p_4,p(BPB=5)=p_5, p(BPB=6)=p_6 \\
Nr &= \mathcal{N}(0, \text{rent model standard error}) \\
Nz &= \mathcal{N}(0, \text{zest model standard error}) \\
Nn &= Bernoulli(p_{south})\\\\
Neigh &= Nn \\
BPB &= Nb \\
Rent &= bpb_3 \cdot a + bpb_4 \cdot b + bpb_5 \cdot d + bpb_6 \cdot d + e +Nrent\\
Zest &= bpb_3 \cdot a + bpb_4 \cdot b + bpb_5 \cdot d + bpb_6 \cdot d + NeighSouth \cdot e + f + Nz \\
ROI &= \text{ROIfunc}(Rent, Zest)
\end{align}

In [9]:
Neigh_alias = ['Neigh_North', 'Neigh_South']
BPB_alias = [2, 3, 4, 5, 6]

south_prob = float(len(df[df['Neigh'] == 'Neigh_South']))/len(df)
bpb_prob = [(df.BPB == a).count()/len(df) for a in BPB_alias]


# Neigh_prob = torch.tensor([north_prob, 1- north_prob])
BPB_prob = torch.tensor(bpb_prob)

exogenous_dists = {
    'Nn': dist.Bernoulli(torch.tensor(south_prob)),
    'Nb': dist.Categorical(BPB_prob),
    'Nr': dist.Normal(torch.tensor(0.), torch.tensor(rent_model_se)),
    'Nz': dist.Normal(torch.tensor(0.), torch.tensor(zest_model_se))
}


def scm(exogenous_dists):
    Nn = pyro.sample("Nn", exogenous_dists['Nn'])
    Nb = pyro.sample("Nb", exogenous_dists['Nb'])
    Nr = pyro.sample("Nr", exogenous_dists['Nr'])
    Nz = pyro.sample("Nz", exogenous_dists['Nz'])

    z_bounds = torch.tensor(zest_model_se)
    r_bounds = torch.tensor(rent_model_se)

    # Stay within 1 SD of 0
    Nz = min(z_bounds, Nz)
    Nz = max(-2 * z_bounds, Nz)

    Nr = min(r_bounds, Nr)
    Nr = max(-2 * r_bounds, Nr)

    Neigh = pyro.sample("Neigh", dist.Delta(Nn))
    BPB = pyro.sample("BPB", dist.Delta(Nb))

    bpb_converted = BPB_alias[BPB.int()]
    neigh_converted = Neigh_alias[Neigh.int()]

    rent_pred = rent_model.predict(pd.DataFrame({'BPB': [bpb_converted]}))[0]
    zest_pred = zest_model.predict(pd.DataFrame(
        {'BPB': [bpb_converted], 'Neigh': [neigh_converted]}))[0]

    Rent = pyro.sample("Rent", dist.Delta(torch.tensor(rent_pred) + Nr))
    Zest = pyro.sample("Zest", dist.Delta(torch.tensor(zest_pred) + Nz))

    ROI = pyro.sample("ROI", dist.Delta(roi(zestimate=Zest,
                                            rental_price=Rent,
                                            inflation_rate=INFLATION_RATE,
                                            mortgage_rate=MORTGAGE_RATE,
                                            num_years=NUM_YEARS,
                                            down_payment_percent=DOWN_PAYMENT)))

    return {'Neigh': Neigh, 'BPB': BPB, 'Rent': Rent, 'Zest': Zest, 'ROI': ROI}

# Model Analysis

### Causal Effect on ROI of Changing Neighborhoods - Should we choose North or South?

$$
P(ROI | do(Neigh=South)) - P(ROI|do(Neigh=North))
$$

In [10]:
south_model = pyro.do(
    model, data={'Neigh': torch.tensor(Neigh_alias.index('Neigh_South'))})
north_model = pyro.do(
    model, data={'Neigh': torch.tensor(Neigh_alias.index('Neigh_North'))})

south_scm = pyro.do(
    scm, data={'Neigh': torch.tensor(Neigh_alias.index('Neigh_South'))})
north_scm = pyro.do(
    scm, data={'Neigh': torch.tensor(Neigh_alias.index('Neigh_North'))})

Causal Effect for Model 2(SCM)

In [11]:
scm_south_roi_samples = [south_scm(exogenous_dists)[
    'ROI'].item() for _ in range(2000)]
scm_north_roi_samples = [north_scm(exogenous_dists)[
    'ROI'].item() for _ in range(2000)]

scm_causal_effect_neigh = mean(
    scm_south_roi_samples) - mean(scm_north_roi_samples)
scm_causal_effect_neigh

0.0061561012649907765

Showing the problematic large variance in the samples

In [12]:
mean(scm_south_roi_samples), np.std(scm_south_roi_samples), min(
    scm_south_roi_samples), max(scm_south_roi_samples)

(-0.9635449091708288, 1.032904101041279, -36.795448303222656, 21.0173282623291)

Causal Effect for Model 1

In [13]:
south_roi_samples = [south_model()['ROI'].item() for _ in range(2000)]
north_roi_samples = [north_model()['ROI'].item() for _ in range(2000)]

causal_effect_neigh = mean(south_roi_samples) - mean(north_roi_samples)
causal_effect_neigh

0.013694935142993914

### Results:
- SCM Model has too much variance and so it makes it difficult to get good samples
- Model 1 shows a 1.3% Increase In ROI from Investing in property in the South as opposed to the North
    - Why?
        - South includes: San Jose, Cupertino and other properties a bit further from SF
        - North includes: Mountain View, Palo Alto - properties in the heart of Silicon Valley and closer to SF

### Causal Effect on ROI of Different BPBs

In [14]:
bpb_roi_samples = []
for i in range(len(BPB_alias)):
    bpb = BPB_alias[i]
    bpb_model = pyro.do(model, data={'BPB': torch.tensor(i)})
    bpb_roi_samples.append([bpb_model()['ROI'].item() for _ in range(2000)])

In [15]:
for i in range(len(BPB_alias)):
    bpb = BPB_alias[i]
    print(bpb, mean(bpb_roi_samples[i]))

2 -0.9815098011493683
3 -0.977315743714571
4 -0.9739876031577587
5 -0.9663166411519051
6 -0.9708639974594117


### What about BPB=5 vs BPB=2 ?
$P(ROI | do(BPB=5)) - P(ROI | do(BPB=2))$

In [16]:
mean(bpb_roi_samples[3]) - mean(bpb_roi_samples[0])

0.01519315999746318

### Results 
- 3 Beds 2 baths is the best configuration for maximizing ROI
- You can increase ROI by 1.5% by doing 3 Beds 2 Baths over a 1 Bed 1 Bath

### Given we want a certain Bed and Baths, where should we buy?

Example for 2 bed 1 bath:
$$
P(ROI | do(Neigh= North), BPB=3) - P(ROI | do(Neigh=South), BPB=3)
$$

In [17]:
for bpb_index in range(len(BPB_alias)):
    bpb = BPB_alias[bpb_index]
    roi_means = []
    cond_model = pyro.condition(model, {"BPB": bpb_index})

    for neigh_index in range(len(Neigh_prob)):
        curr_model = pyro.do(cond_model, {'Neigh': neigh_index})
        roi_mean = curr_model()['ROI'].item()
        roi_means.append(roi_mean)

    print("𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑁𝑜𝑟𝑡ℎ),𝐵𝑃𝐵=%d)−𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑆𝑜𝑢𝑡ℎ),𝐵𝑃𝐵=%d) = %.3f - %.3f = %.3f" % (bpb, bpb,
                                                                                                roi_means[0],
                                                                                                roi_means[1],
                                                                                                roi_means[0] -
                                                                                                roi_means[1]
                                                                                                ))

𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑁𝑜𝑟𝑡ℎ),𝐵𝑃𝐵=2)−𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑆𝑜𝑢𝑡ℎ),𝐵𝑃𝐵=2) = -0.985 - -0.976 = -0.009
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑁𝑜𝑟𝑡ℎ),𝐵𝑃𝐵=3)−𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑆𝑜𝑢𝑡ℎ),𝐵𝑃𝐵=3) = -0.983 - -0.970 = -0.013
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑁𝑜𝑟𝑡ℎ),𝐵𝑃𝐵=4)−𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑆𝑜𝑢𝑡ℎ),𝐵𝑃𝐵=4) = -0.980 - -0.966 = -0.014
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑁𝑜𝑟𝑡ℎ),𝐵𝑃𝐵=5)−𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑆𝑜𝑢𝑡ℎ),𝐵𝑃𝐵=5) = -0.974 - -0.956 = -0.019
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑁𝑜𝑟𝑡ℎ),𝐵𝑃𝐵=6)−𝑃(𝑅𝑂𝐼|𝑑𝑜(𝑁𝑒𝑖𝑔ℎ=𝑆𝑜𝑢𝑡ℎ),𝐵𝑃𝐵=6) = -0.977 - -0.964 = -0.013


### Results
- No matter which BPB you are looking at, it is always better to pick a property in the South rather than the North

### Given we want to buy in a certain area, what beds and baths should we buy?

Example for the North Neighborhood
$$
P(ROI | do(BPB= x), Neigh= North) - P(ROI | do(BPB=y), Neigh= North)
$$

In [18]:
for neigh_index in range(len(Neigh_prob)):
    roi_means = []
    cond_model = pyro.condition(model, {"Neigh": neigh_index})
    neigh = Neigh_alias[neigh_index]
    print(neigh)

    for bpb_index in range(len(BPB_prob)):
        bpb = BPB_alias[bpb_index]
        curr_model = pyro.do(cond_model, {'BPB': bpb_index})
        roi_mean = curr_model()['ROI'].item()
        print("𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=%s),𝑁𝑒𝑖𝑔ℎ=%s) = %.3f" % (bpb, neigh, roi_mean))

Neigh_North
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=2),𝑁𝑒𝑖𝑔ℎ=Neigh_North) = -0.985
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=3),𝑁𝑒𝑖𝑔ℎ=Neigh_North) = -0.983
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=4),𝑁𝑒𝑖𝑔ℎ=Neigh_North) = -0.980
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=5),𝑁𝑒𝑖𝑔ℎ=Neigh_North) = -0.974
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=6),𝑁𝑒𝑖𝑔ℎ=Neigh_North) = -0.977
Neigh_South
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=2),𝑁𝑒𝑖𝑔ℎ=Neigh_South) = -0.976
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=3),𝑁𝑒𝑖𝑔ℎ=Neigh_South) = -0.970
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=4),𝑁𝑒𝑖𝑔ℎ=Neigh_South) = -0.966
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=5),𝑁𝑒𝑖𝑔ℎ=Neigh_South) = -0.956
𝑃(𝑅𝑂𝐼|𝑑𝑜(𝐵𝑃𝐵=6),𝑁𝑒𝑖𝑔ℎ=Neigh_South) = -0.964


### Results
- If you are interested in purchasing a property in the North, it should be a 3 bed 2 bath. If you are interested in purchasing a property in the South it should also be a 3 bed 2 bath.

# Property Examples
- here are the properties with the highest ROIs
- We can see that these properties follow the effects we have seen in our model
    - 5 BPB, all are South Neighborhood

In [19]:
df.sort_values(["ROI"], ascending=False)[:5]

,Beds,Baths,Rent,neighbourhood,latitude,longitude,Type,Zest,ROI,Neigh,BPB,latlng
27,2,2,140.0,San Jose,37.40075,-121.92194,Townhouse,214665.0,-0.883995,Neigh_South,4,-4559.971997
110,3,2,550.0,San Jose,37.31737,-121.78764,Townhouse,998982.0,-0.902070,Neigh_South,5,-4544.794423
115,2,2,275.0,San Jose,37.31392,-121.93997,Condominium,513454.0,-0.904733,Neigh_South,4,-4550.058285
16,2,2,499.0,San Jose,37.31996,-121.94828,Condominium,1210000.0,-0.926646,Neigh_South,4,-4551.104932
17,3,2,375.0,San Jose,37.39220,-121.86552,Condominium,930000.0,-0.928277,Neigh_South,5,-4556.819897


- [Property 1](https://www.zillow.com/homedetails/435-Milan-Dr-San-Jose-CA-95134/2143393753_zpid/)
- [Property 2](https://www.zillow.com/homedetails/3215-Lamond-Ct-San-Jose-CA-95148/19793643_zpid/)
- [Property 5](https://www.zillow.com/homedetails/2409-Venturi-Pl-UNIT-2-San-Jose-CA-95132/124743156_zpid/)

# Next Steps

1. **Get more data**
2. Consider adding new nodes to the DAG
    1. Individual Neighborhoods as opposed to North and South
    2. Host and ratings
    3. Distance to transit
3. Reconsider new edges with our new data
3. Test more complex functions to fit Rent and Zestimate
    1. Currently using factor effects models
4. Attempt to use SCM again
    1. Our models have high Std Error so using SCM is difficult
    2. Increasing the amount of data would also decrease Std. Error and help give better results from SCM